In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 29 08:30:42 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%capture
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install evaluate
import os
import re
import wandb
import torch
import numpy as np
import json
from evaluate import load
from datasets import load_dataset, Audio
from transformers import EarlyStoppingCallback
from huggingface_hub import HfFolder


In [ ]:
import wandb
import torch
import numpy as np
import json
from datasets import load_dataset,  Audio
from transformers import (     Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,
    Wav2Vec2ForCTC, TrainingArguments, Trainer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
# from dotenv import load_dotenv
from transformers import EarlyStoppingCallback
from huggingface_hub import login

In [ ]:
HF_API_TOKEN = os.getenv("HF_TOKEN")  # <-- Use environment variable for Hugging Face tokenSXwdwQiQBwpzvZdjPObRIxFxMSQWspDVFJ"
WANDB_API_KEY = "796635845bbcb935e270354f88bafb37436ad1c4"
# Load datasets
# Login to WandB and Hugging Face
wandb.login(key=WANDB_API_KEY)
login(HF_API_TOKEN)

wandb: Currently logged in as: kyagabajonah (asr-africa-research-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# **AMHARIC**

In [ ]:
from datasets import load_dataset, DatasetDict

test5hr = load_dataset("KYAGABA/kin_cleaned_commonvoice")['train']
print(test5hr)
test5hr=test5hr.cast_column("audio", Audio(sampling_rate=16_000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/741 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/73 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/73 [00:00<?, ?it/s]

train-00000-of-00073.parquet:   0%|          | 0.00/549M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/229498 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14396 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/14447 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/71 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'sentence', 'audio_len', 'transcript_len', 'len_ratio', 'num_feature_vecs'],
    num_rows: 229498
})


In [ ]:
SAMPLE_RATE=16000
total_duration_train = sum(len(example["audio"]["array"]) for example in test5hr) / SAMPLE_RATE / 3600
total_duration_train

312.47334

In [ ]:
import unicodedata
import re
def preprocess_text(batch):
    batch['sentence'] = [text.lower() for text in batch['sentence']]
    batch['sentence'] = [unicodedata.normalize('NFKC', text) for text in batch['sentence']]
    batch['sentence'] = [re.sub(r'[\’\ʻ\ʼ\ʽ\‘\']', '', text) for text in batch['sentence']]
    batch['sentence'] = [re.sub(r'[^\w\s\']', '', text) for text in batch['sentence']]
    batch['sentence'] = [re.sub(r'_', ' ', text) for text in batch['sentence']]
    batch['sentence'] = [' '.join(text.split()) for text in batch['sentence']]
    return batch

test5hr = test5hr.map(preprocess_text, batched=True)

Map:   0%|          | 0/229498 [00:00<?, ? examples/s]

OSError: [Errno 28] No space left on device

In [ ]:
def to_lowercase(example):
    example["sentence"] = example["sentence"].lower()
    return example

test5hr = test5hr.map(to_lowercase)

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = test5hr.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test5hr.column_names)
vocab_list = list(set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

In [ ]:
# models = [
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_1hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_5hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_10hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_NCHLT_Speech_corpus_zulu_50hr_v2",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
# ]

In [ ]:
# models = [
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_1hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_5hr_v3",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_10hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_50hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
# ]


In [ ]:
# models = [
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_1hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_1hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_5hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_5hr_v3",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_10hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_10hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_NCHLT_Speech_corpus_zulu_50hr_v2",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_50hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
# ]


In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = [
    # "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2",
    # "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2",
    # "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3",
    # "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-20hrs-3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs"


]


In [ ]:
import jiwer

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_errors(prediction, reference):
    # Compute alignment between prediction and reference
    alignment = jiwer.process_words(reference, prediction)

    # Extract substitution, insertion, and deletion counts
    substitutions = alignment.substitutions
    insertions = alignment.insertions
    deletions = alignment.deletions

    # Total words in the reference
    total_words = len(reference.split())

    return {
        "substitutions": substitutions,
        "insertions": insertions,
        "deletions": deletions,
        "total_words": total_words,
    }

In [ ]:
def compute_errors(prediction, reference):
    ref_words = reference.split()
    pred_words = prediction.split()

    # Initialize a matrix for Levenshtein alignment
    dp = [[0] * (len(pred_words) + 1) for _ in range(len(ref_words) + 1)]

    for i in range(len(ref_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            else:
                if ref_words[i - 1] == pred_words[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    # Backtrack to count substitutions, insertions, and deletions
    i, j = len(ref_words), len(pred_words)
    substitutions, insertions, deletions = 0, 0, 0

    while i > 0 and j > 0:
        if ref_words[i - 1] == pred_words[j - 1]:
            i -= 1
            j -= 1
        else:
            if dp[i][j] == dp[i - 1][j - 1] + 1:
                substitutions += 1
                i -= 1
                j -= 1
            elif dp[i][j] == dp[i - 1][j] + 1:
                deletions += 1
                i -= 1
            else:
                insertions += 1
                j -= 1

    while i > 0:
        deletions += 1
        i -= 1
    while j > 0:
        insertions += 1
        j -= 1

    return {

        "substitutions": substitutions,
        "insertions": insertions,
        "deletions": deletions,
        "total_words": len(ref_words),
    }

In [ ]:
def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    errors = compute_errors(prediction, reference)
    return wer, cer, errors

In [ ]:
from tqdm import tqdm
from transformers import pipeline
import pandas as pd

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device, return_timestamps='word')
    results = []
    all_predictions = []
    all_references = []
    total_substitutions, total_insertions, total_deletions, total_words = 0, 0, 0, 0

    for audio, sentence in tqdm(zip(test5hr["audio"], test5hr["sentence"]),
                                     total=test5hr.num_rows, desc="Running inference"):
        # Run inference
        prediction = pipe(audio)['text']

        # Compute metrics
        wer, cer, errors = compute_individual_metrics(prediction, sentence)

        # Accumulate errors
        total_substitutions += errors["substitutions"]
        total_insertions += errors["insertions"]
        total_deletions += errors["deletions"]
        total_words += errors["total_words"]

        # Append results
        results.append({
            "Prediction": prediction,
            "Reference": sentence,
            "WER": wer,
            "CER": cer,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })
        all_predictions.append(prediction)
        all_references.append(sentence)

    # Create results DataFrame
    df = pd.DataFrame(results)

    # Compute overall metrics
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    # Print overall metrics
    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    print("Total Substitutions:", total_substitutions)
    print("Total Insertions:", total_insertions)
    print("Total Deletions:", total_deletions)
    print("Total Words:", total_words)
    print("Substitution Rate:", total_substitutions / total_words)
    print("Insertion Rate:", total_insertions / total_words)
    print("Deletion Rate:", total_deletions / total_words)

    # Save detailed results to CSV
    output_filename = f'sentences_{model_name.split("/")[-1]}.csv'
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")


In [ ]:
for model in models:
    run_inference_for_model(model)

Processing model: asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3


config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

Running inference: 100%|██████████| 3163/3163 [10:49<00:00,  4.87it/s]


Calculated overall WER: 0.3199705489139912
Calculated overall CER: 0.1089415118995763
Total Substitutions: 13214
Total Insertions: 2137
Total Deletions: 1163
Total Words: 51611
Substitution Rate: 0.2560306911317355
Insertion Rate: 0.041405901842630446
Deletion Rate: 0.022533955939625275
Results saved to transcriptions_wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs.csv


In [ ]:
from collections import Counter

# Function to find common errors
def find_common_errors(df, error_type="Substitutions"):
    errors = []
    for _, row in df.iterrows():
        ref_words = row["Reference"].split()
        pred_words = row["Prediction"].split()
        if error_type == "Substitutions":
            for ref, pred in zip(ref_words, pred_words):
                if ref != pred:
                    errors.append((ref, pred))
        elif error_type == "Insertions":
            if len(pred_words) > len(ref_words):
                for word in pred_words:
                    if word not in ref_words:
                        errors.append(word)
        elif error_type == "Deletions":
            if len(ref_words) > len(pred_words):
                for word in ref_words:
                    if word not in pred_words:
                        errors.append(word)
    return Counter(errors).most_common(10)

# Most common substitutions
print("Most Common Substitutions:")
print(find_common_errors(df, error_type="Substitutions"))

# Most common insertions
print("Most Common Insertions:")
print(find_common_errors(df, error_type="Insertions"))

# Most common deletions
print("Most Common Deletions:")
print(find_common_errors(df, error_type="Deletions"))

Most Common Substitutions:


NameError: name 'df' is not defined

In [ ]:
!pip install jiwer pandas

import pandas as pd
import numpy as np
from collections import defaultdict
import jiwer
import unicodedata

########################################
# 1. Helper: Normalize Amharic
########################################

def normalize_amharic(text):
    """
    Example normalization function for Amharic.
    Adjust as needed (remove or unify diacritics, punctuation, etc.).
    """
    return unicodedata.normalize("NFC", str(text))

########################################
# 2. Custom Alignment (Word-Level)
########################################

def align_tokens(ref_tokens, hyp_tokens):
    """
    Perform a simple dynamic programming alignment (Levenshtein).
    Returns a list of (ref_word, hyp_word, operation):
      operation ∈ {"hit", "sub", "ins", "del"}
    """
    n = len(ref_tokens)
    m = len(hyp_tokens)

    # dp[i][j] = (cost, alignment_list)
    dp = [[(0, []) for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize first column (all deletions)
    for i in range(1, n + 1):
        cost_prev, align_prev = dp[i - 1][0]
        dp[i][0] = (
            cost_prev + 1,
            align_prev + [(ref_tokens[i - 1], "", "del")]
        )

    # Initialize first row (all insertions)
    for j in range(1, m + 1):
        cost_prev, align_prev = dp[0][j - 1]
        dp[0][j] = (
            cost_prev + 1,
            align_prev + [("", hyp_tokens[j - 1], "ins")]
        )

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ref_word = ref_tokens[i - 1]
            hyp_word = hyp_tokens[j - 1]

            # If they match, no cost for this token
            if ref_word == hyp_word:
                cost_prev, align_prev = dp[i - 1][j - 1]
                dp[i][j] = (cost_prev, align_prev + [(ref_word, hyp_word, "hit")])
            else:
                # Substitution
                sub_cost, sub_align = dp[i - 1][j - 1]
                sub_cost += 1
                sub_align = sub_align + [(ref_word, hyp_word, "sub")]

                # Insertion
                ins_cost, ins_align = dp[i][j - 1]
                ins_cost += 1
                ins_align = ins_align + [("", hyp_word, "ins")]

                # Deletion
                del_cost, del_align = dp[i - 1][j]
                del_cost += 1
                del_align = del_align + [(ref_word, "", "del")]

                # Choose minimal cost
                if sub_cost <= ins_cost and sub_cost <= del_cost:
                    dp[i][j] = (sub_cost, sub_align)
                elif ins_cost <= del_cost:
                    dp[i][j] = (ins_cost, ins_align)
                else:
                    dp[i][j] = (del_cost, del_align)

    # Final alignment is in dp[n][m]
    _, alignment_list = dp[n][m]
    return alignment_list

########################################
# 3. Metrics + Alignment
########################################

def compute_individual_metrics(hypothesis, reference):
    """
    Compute WER, CER, error counts, plus alignment.
    Returns:
      wer_val, cer_val, errors, alignment
    """
    ref_norm = normalize_amharic(reference)
    hyp_norm = normalize_amharic(hypothesis)

    # Use jiwer for WER, CER
    measures = jiwer.compute_measures(ref_norm, hyp_norm)
    wer_val  = measures.get("wer", 0.0)
    sub      = measures.get("substitutions", 0)
    ins      = measures.get("insertions", 0)
    dele     = measures.get("deletions", 0)
    total_words = len(ref_norm.split())
    errors = {
        "substitutions": sub,
        "insertions": ins,
        "deletions": dele,
        "total_words": total_words
    }

    cer_val = jiwer.cer(ref_norm, hyp_norm)

    # Word-level alignment
    ref_tokens = ref_norm.split()
    hyp_tokens = hyp_norm.split()
    alignment  = align_tokens(ref_tokens, hyp_tokens)

    return wer_val, cer_val, errors, alignment

########################################
# 4. Optional: Character-Level Difference
########################################

def compare_characters(ref_word, hyp_word):
    """
    Returns a list of (ref_char, hyp_char) pairs
    for each position. If lengths differ, extra chars
    are paired with ''.
    """
    max_len = max(len(ref_word), len(hyp_word))
    pairs = []
    for i in range(max_len):
        r_char = ref_word[i] if i < len(ref_word) else ""
        h_char = hyp_word[i] if i < len(hyp_word) else ""
        if r_char != h_char:
            pairs.append((r_char, h_char))
    return pairs

########################################
# 5. Analyze Substitution Context
########################################

def analyze_substitution_contexts(substitution_list, top_n=10):
    """
    - substitution_list: list of dict with keys:
        {
          "preceding_context": list[str],
          "ref_word": str,
          "hyp_word": str,
          "next_word": str or None
        }
    - top_n: how many top patterns to print
    """
    # (ref_word -> hyp_word) counts
    sub_counts = defaultdict(int)
    # preceding context counts
    context_counts = defaultdict(int)

    for item in substitution_list:
        ref_word  = item["ref_word"]
        hyp_word  = item["hyp_word"]
        ctx_tuple = tuple(item["preceding_context"])
        sub_counts[(ref_word, hyp_word)] += 1
        context_counts[ctx_tuple]        += 1

    # Show top N frequent substitutions
    sorted_sub_counts = sorted(sub_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Word Patterns:")
    for (ref, hyp), count in sorted_sub_counts[:top_n]:
        print(f"  '{ref}' -> '{hyp}': {count} times")

    # Show top N preceding contexts
    sorted_ctx_counts = sorted(context_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Preceding Contexts Causing Substitutions:")
    for ctx, count in sorted_ctx_counts[:top_n]:
        print(f"  Context {ctx}: {count} times")

########################################
# 6. Main Analysis Function
########################################

def analyze_csv(
    csv_file,
    output_analyzed="results_analyzed.csv",
    output_sub_contexts="substitution_contexts.csv",
    do_char_diff=False
):
    """
    - csv_file: path to CSV with columns ["Prediction", "Reference"].
    - output_analyzed: row-level metrics output CSV
    - output_sub_contexts: bigram/trigram context output CSV
    - do_char_diff: if True, store character-level diffs for each substitution

    Steps:
    1) read CSV
    2) for each row => compute WER, CER, alignment
    3) gather substitution contexts
    4) write row-level results to <output_analyzed>
    5) analyze + write aggregated contexts to <output_sub_contexts>
    """
    df = pd.read_csv(csv_file)
    results_rows = []
    substitution_list = []

    total_subs = 0
    total_ins  = 0
    total_del  = 0
    total_wds  = 0

    # Process each row
    for idx, row in df.iterrows():
        pred = str(row["Prediction"])
        ref  = str(row["Reference"])

        wer_val, cer_val, errors, alignment = compute_individual_metrics(pred, ref)
        total_subs += errors["substitutions"]
        total_ins  += errors["insertions"]
        total_del  += errors["deletions"]
        total_wds  += errors["total_words"]

        # gather bigram/trigram context for each sub
        ref_tokens = [a[0] for a in alignment]  # after alignment
        hyp_tokens = [a[1] for a in alignment]
        ops        = [a[2] for a in alignment]

        # For each alignment step, if 'sub', collect context
        # We'll do 2 preceding words + the next word (trigram).
        for i, op_type in enumerate(ops):
            if op_type == "sub":
                start_idx = max(0, i - 2)   # up to 2 preceding
                preceding_context = [t for t in ref_tokens[start_idx:i] if t]

                ref_word = ref_tokens[i]
                hyp_word = hyp_tokens[i]
                next_word = ref_tokens[i + 1] if (i + 1) < len(ref_tokens) else None

                # optional character-level analysis
                char_diffs = compare_characters(ref_word, hyp_word) if do_char_diff else []

                substitution_list.append({
                    "row_idx": idx,
                    "preceding_context": preceding_context,
                    "ref_word": ref_word,
                    "hyp_word": hyp_word,
                    "next_word": next_word,
                    "char_diffs": char_diffs
                })

        results_rows.append({
            "Prediction": pred,
            "Reference": ref,
            "WER": wer_val,
            "CER": cer_val,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })

    results_df = pd.DataFrame(results_rows)

    # Overall metrics
    overall_wer = jiwer.wer(list(df["Reference"]), list(df["Prediction"]))
    overall_cer = jiwer.cer(list(df["Reference"]), list(df["Prediction"]))

    print("\n=== Overall Metrics ===")
    print(f"Calculated overall WER: {overall_wer}")
    print(f"Calculated overall CER: {overall_cer}")
    print(f"Total Substitutions: {total_subs}")
    print(f"Total Insertions: {total_ins}")
    print(f"Total Deletions: {total_del}")
    print(f"Total Words: {total_wds}")
    if total_wds > 0:
        print(f"Substitution Rate: {total_subs / total_wds}")
        print(f"Insertion Rate: {total_ins / total_wds}")
        print(f"Deletion Rate: {total_del / total_wds}")

    results_df.to_csv(output_analyzed, index=False)
    print(f"\nRow-level results saved to: {output_analyzed}")

    # Now let's do aggregated substitution analysis
    analyze_substitution_contexts(substitution_list, top_n=10)

    # Optionally write out the list of all substitution contexts
    sub_df = pd.DataFrame(substitution_list)
    sub_df.to_csv(output_sub_contexts, index=False)
    print(f"Detailed substitution contexts saved to: {output_sub_contexts}")

##################################
# Example usage in Colab
##################################
csv_path = "sentences_wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs.csv"

# Turn on do_char_diff if you want per-substitution character analysis
analyze_csv(csv_file=csv_path,
            output_analyzed="my_amharic_analyzed.csv",
            output_sub_contexts="my_subcontexts.csv",
            do_char_diff=True)



=== Overall Metrics ===
Calculated overall WER: 0.3199705489139912
Calculated overall CER: 0.11002653150120778
Total Substitutions: 13098
Total Insertions: 2195
Total Deletions: 1221
Total Words: 51611
Substitution Rate: 0.2537831082521168
Insertion Rate: 0.04252969328243979
Deletion Rate: 0.023657747379434615

Row-level results saved to: my_amharic_analyzed.csv

Top 10 Substitution Word Patterns:
  'እና' -> 'ና': 74 times
  'እና' -> 'እ': 27 times
  'እናም' -> 'እና': 14 times
  'ወይም' -> 'ወም': 11 times
  'ሚሊዮን' -> 'ሚሊየን': 9 times
  'ወይም' -> 'ም': 9 times
  'አለብዎት' -> 'አለቦት': 8 times
  'ይችላል' -> 'ይችላሉ': 8 times
  'ሀብረት' -> 'ህብረት': 8 times
  'ሰዎችን' -> 'ሰዎች': 8 times

Top 10 Preceding Contexts Causing Substitutions:
  Context (): 1104 times
  Context ('በ',): 61 times
  Context ('እና',): 41 times
  Context ('ውስጥ',): 25 times
  Context ('ከ',): 22 times
  Context ('የ',): 17 times
  Context ('ነገር', 'ግን'): 16 times
  Context ('ይህ',): 16 times
  Context ('ወደ',): 16 times
  Context ('ብዙውን', 'ጊዜ'): 15 ti

In [ ]:
!pip install jiwer pandas

import pandas as pd
import numpy as np
from collections import defaultdict
import jiwer
import unicodedata

########################################
# 1. Helper: Normalize Amharic
########################################

def normalize_amharic(text):
    """
    Example normalization function for Amharic.
    Adjust as needed (remove or unify diacritics, punctuation, etc.).
    """
    return unicodedata.normalize("NFC", str(text))

########################################
# 2. Custom Alignment (Word-Level)
########################################

def align_tokens(ref_tokens, hyp_tokens):
    """
    Perform a simple dynamic programming alignment (Levenshtein).
    Returns a list of (ref_word, hyp_word, operation):
      operation ∈ {"hit", "sub", "ins", "del"}
    """
    n = len(ref_tokens)
    m = len(hyp_tokens)

    # dp[i][j] = (cost, alignment_list)
    dp = [[(0, []) for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize first column (all deletions)
    for i in range(1, n + 1):
        cost_prev, align_prev = dp[i - 1][0]
        dp[i][0] = (
            cost_prev + 1,
            align_prev + [(ref_tokens[i - 1], "", "del")]
        )

    # Initialize first row (all insertions)
    for j in range(1, m + 1):
        cost_prev, align_prev = dp[0][j - 1]
        dp[0][j] = (
            cost_prev + 1,
            align_prev + [("", hyp_tokens[j - 1], "ins")]
        )

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ref_word = ref_tokens[i - 1]
            hyp_word = hyp_tokens[j - 1]

            # If they match, no cost for this token
            if ref_word == hyp_word:
                cost_prev, align_prev = dp[i - 1][j - 1]
                dp[i][j] = (cost_prev, align_prev + [(ref_word, hyp_word, "hit")])
            else:
                # Substitution
                sub_cost, sub_align = dp[i - 1][j - 1]
                sub_cost += 1
                sub_align = sub_align + [(ref_word, hyp_word, "sub")]

                # Insertion
                ins_cost, ins_align = dp[i][j - 1]
                ins_cost += 1
                ins_align = ins_align + [("", hyp_word, "ins")]

                # Deletion
                del_cost, del_align = dp[i - 1][j]
                del_cost += 1
                del_align = del_align + [(ref_word, "", "del")]

                # Choose minimal cost
                if sub_cost <= ins_cost and sub_cost <= del_cost:
                    dp[i][j] = (sub_cost, sub_align)
                elif ins_cost <= del_cost:
                    dp[i][j] = (ins_cost, ins_align)
                else:
                    dp[i][j] = (del_cost, del_align)

    # Final alignment is in dp[n][m]
    _, alignment_list = dp[n][m]
    return alignment_list

########################################
# 3. Levenshtein Edit Distance (Char-Level)
########################################

def levenshtein_distance(a, b):
    """
    Simple DP approach to compute edit distance between two strings a, b.
    """
    A, B = len(a), len(b)
    dp = np.zeros((A + 1, B + 1), dtype=int)
    for i in range(A + 1):
        dp[i][0] = i
    for j in range(B + 1):
        dp[0][j] = j

    for i in range(1, A + 1):
        for j in range(1, B + 1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,     # deletion
                dp[i][j-1] + 1,     # insertion
                dp[i-1][j-1] + cost # substitution
            )
    return dp[A][B]

########################################
# 4. Morphological "Root" (Placeholder)
########################################

def get_morphological_root(amharic_word):
    """
    Naive placeholder for morphological analysis.
    Removes common suffixes, etc.
    You can integrate a real morphological analyzer here.

    Example suffixes (very naive):
      - ም, ማ, ች, ቹ, ው, ዎች, ነ, etc.

    NOTE: This is a simplistic example.
    Customize or replace with a real parser for better results.
    """
    # Common suffixes you want to strip out
    suffixes = ["ም", "ማ", "ች", "ቹ", "ው", "ዎች", "ነ", "ዋ", "ቷ", "ሗ", "ሉ"]
    root = amharic_word
    for sfx in suffixes:
        if root.endswith(sfx) and len(root) > len(sfx):
            root = root[: -len(sfx)]
    return root

########################################
# 5. Main Metrics + Substitution Extraction
########################################

def compute_individual_metrics(hypothesis, reference):
    """
    Compute WER, CER, error counts, plus alignment.
    Returns:
      wer_val, cer_val, errors, alignment
    """
    ref_norm = normalize_amharic(reference)
    hyp_norm = normalize_amharic(hypothesis)

    # Use jiwer for WER, CER
    measures = jiwer.compute_measures(ref_norm, hyp_norm)
    wer_val  = measures.get("wer", 0.0)
    sub      = measures.get("substitutions", 0)
    ins      = measures.get("insertions", 0)
    dele     = measures.get("deletions", 0)
    total_words = len(ref_norm.split())
    errors = {
        "substitutions": sub,
        "insertions": ins,
        "deletions": dele,
        "total_words": total_words
    }

    cer_val = jiwer.cer(ref_norm, hyp_norm)

    # Word-level alignment
    ref_tokens = ref_norm.split()
    hyp_tokens = hyp_norm.split()
    alignment  = align_tokens(ref_tokens, hyp_tokens)

    return wer_val, cer_val, errors, alignment

########################################
# 6. Comprehensive Substitution Analysis
########################################

def analyze_substitutions_deeper(substitution_list, top_n=10):
    """
    For each item in substitution_list (which has keys like):
      {
        "preceding_context": [...],
        "ref_word": str,
        "hyp_word": str,
        "char_distance": int (Levenshtein distance),
        "ref_root": str,
        "hyp_root": str
      }
    we do:

      1. Print top-n frequent (ref_word -> hyp_word) pairs
      2. Print top-n frequent preceding contexts
      3. Print top-n frequent root pairs
      4. Print top-n frequent character mismatches across the entire dataset
         (which glyph is replaced by which glyph)
    """

    # 1) Count (ref_word -> hyp_word)
    pair_counts = defaultdict(int)
    # 2) preceding context
    context_counts = defaultdict(int)
    # 3) (ref_root -> hyp_root)
    root_counts = defaultdict(int)
    # 4) track all character pairs (char -> char)
    char_pair_counts = defaultdict(int)

    for item in substitution_list:
        r_w = item["ref_word"]
        h_w = item["hyp_word"]
        ctx_tuple = tuple(item["preceding_context"])
        r_root = item["ref_root"]
        h_root = item["hyp_root"]
        # gather char-level mismatches
        for (r_char, h_char) in item["char_mismatches"]:
            char_pair_counts[(r_char, h_char)] += 1

        pair_counts[(r_w, h_w)] += 1
        context_counts[ctx_tuple] += 1
        root_counts[(r_root, h_root)] += 1

    # Print top-n frequent (ref->hyp)
    sorted_pair_counts = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Word Patterns:")
    for (r, h), cnt in sorted_pair_counts[:top_n]:
        print(f"  '{r}' -> '{h}': {cnt} times")

    # Print top-n preceding contexts
    sorted_ctx_counts = sorted(context_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Preceding Contexts Causing Substitutions:")
    for ctx, cnt in sorted_ctx_counts[:top_n]:
        print(f"  Context {ctx}: {cnt} times")

    # Print top-n root pairs
    sorted_root_counts = sorted(root_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Root Patterns:")
    for (r_root, h_root), cnt in sorted_root_counts[:top_n]:
        # skip trivial empty roots if any
        if not r_root and not h_root:
            continue
        print(f"  '{r_root}' -> '{h_root}': {cnt} times")

    # Print top-n character mismatch pairs
    sorted_char_pairs = sorted(char_pair_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Character Mismatch Pairs (Across All Subs):")
    for (r_c, h_c), cnt in sorted_char_pairs[:top_n]:
        # show them as 'እ'->'ኧ' : 12 times
        print(f"  '{r_c}' -> '{h_c}': {cnt} times")

########################################
# 7. Main "analyze_csv" function
########################################

def analyze_csv(csv_file, top_n=10):
    """
    - csv_file: path to CSV with columns ["Prediction", "Reference"].
    - We do a deeper analysis:
      (a) WER/CER row-level
      (b) Substitution context extraction (with bigram preceding words)
      (c) Morphological "root" + character-level edit distance
      (d) Summaries all displayed in console (no extra CSV).
    """
    df = pd.read_csv(csv_file)
    results_rows = []
    substitution_list = []

    total_subs = 0
    total_ins  = 0
    total_del  = 0
    total_wds  = 0

    # Process each row
    for idx, row in df.iterrows():
        pred = str(row["Prediction"])
        ref  = str(row["Reference"])

        wer_val, cer_val, errors, alignment = compute_individual_metrics(pred, ref)
        total_subs += errors["substitutions"]
        total_ins  += errors["insertions"]
        total_del  += errors["deletions"]
        total_wds  += errors["total_words"]

        # gather bigram/trigram context for each sub
        ref_tokens = [a[0] for a in alignment]
        hyp_tokens = [a[1] for a in alignment]
        ops        = [a[2] for a in alignment]

        for i, op_type in enumerate(ops):
            if op_type == "sub":
                start_idx = max(0, i - 2)
                preceding_context = [t for t in ref_tokens[start_idx:i] if t]

                ref_word = ref_tokens[i]
                hyp_word = hyp_tokens[i]
                next_word = ref_tokens[i + 1] if (i + 1) < len(ref_tokens) else None

                # Compute character-level edit distance & mismatches
                char_dist = levenshtein_distance(ref_word, hyp_word)
                # figure out character mismatches quickly
                # naive approach: pair up each position, ignoring alignment
                # or do a quick approach:
                max_len = max(len(ref_word), len(hyp_word))
                char_mismatches = []
                for c_idx in range(max_len):
                    r_c = ref_word[c_idx] if c_idx < len(ref_word) else ""
                    h_c = hyp_word[c_idx] if c_idx < len(hyp_word) else ""
                    if r_c != h_c:
                        char_mismatches.append((r_c, h_c))

                # morphological root extraction
                ref_root = get_morphological_root(ref_word)
                hyp_root = get_morphological_root(hyp_word)

                substitution_list.append({
                    "row_idx": idx,
                    "preceding_context": preceding_context,
                    "ref_word": ref_word,
                    "hyp_word": hyp_word,
                    "next_word": next_word,
                    "char_distance": char_dist,
                    "char_mismatches": char_mismatches,
                    "ref_root": ref_root,
                    "hyp_root": hyp_root
                })

        results_rows.append({
            "Prediction": pred,
            "Reference": ref,
            "WER": wer_val,
            "CER": cer_val,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })

    results_df = pd.DataFrame(results_rows)

    # Overall metrics
    # We'll also do a final pass with jiwer to confirm
    overall_wer = jiwer.wer(list(df["Reference"]), list(df["Prediction"]))
    overall_cer = jiwer.cer(list(df["Reference"]), list(df["Prediction"]))

    print("\n=== Overall Metrics ===")
    print(f"Calculated overall WER: {overall_wer}")
    print(f"Calculated overall CER: {overall_cer}")
    print(f"Total Substitutions: {total_subs}")
    print(f"Total Insertions: {total_ins}")
    print(f"Total Deletions: {total_del}")
    print(f"Total Words: {total_wds}")
    if total_wds > 0:
        print(f"Substitution Rate: {total_subs / total_wds}")
        print(f"Insertion Rate: {total_ins / total_wds}")
        print(f"Deletion Rate: {total_del / total_wds}")

    # Print row-level summary (first few rows) if desired
    print("\n=== Sample of Row-Level Results ===")
    print(results_df.head(5))

    # Deeper analysis: morphological root clustering, char-level mismatch stats, etc.
    analyze_substitutions_deeper(substitution_list, top_n=top_n)

    # Show sample from substitution_list
    print("\n=== Sample of Substitution Contexts (mysubcontexts) ===")
    for item in substitution_list[:top_n]:
        print(item)

########################################
# 8. Example usage
########################################
csv_path = "sentences_wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs.csv"
analyze_csv(csv_file=csv_path, top_n=10)



=== Overall Metrics ===
Calculated overall WER: 0.3199705489139912
Calculated overall CER: 0.11002653150120778
Total Substitutions: 13098
Total Insertions: 2195
Total Deletions: 1221
Total Words: 51611
Substitution Rate: 0.2537831082521168
Insertion Rate: 0.04252969328243979
Deletion Rate: 0.023657747379434615

=== Sample of Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐርማርኬት መግዛት በተለምዶ ርካሹ ራስን የመመገበ ዘዴ ነው ምግ...   
1  ጃስሰይ ሪፐን በማኮብፖበ ሜዳ ላይ በኝ ተኩል ኤኤም አከባቢያዊ ሰአት በዜ...   
2  ሰው አልባው የጨረቃ ተላይትቻንድሪያን አንድ የሞ ኢንፓክት ፒሮቡን ወይም ...   
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...   
4     ሶውዬት ዩኒን ትልቅ ክብር የሶቬት ዩኒየን ጀግና ለስራው ስራ ሁሉ ተሰጠው   

                                           Reference       WER       CER  \
0  ምግብ ከሱፐርማርኬት መግዛት በተለምዶ ርካሹ እራስን የመመገቢያ ዘዴ ነው ...  0.238095  0.057692   
1  በማኮብኮብያ ሜዳ ላይ በ አካባቢያዊ ሰእት ተከሰከሰ እና ፈነዳ ስለሆነም ...  0.764706  0.506329   
2  ሰው አልባው የጨረቃ ሳተላይት ቻንድራያን የሙን ኢምፓክት ፕሮቡን ኤምኢይፒ...  0.640000  0.303279   
3  ቅፅበታዊ 